In [1]:
import cv2
import itertools
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def detectFacialLandmarks(image, face_mesh):
    '''
    This function performs facial landmarks detection on an image.
    Args:
        image:     The input image of person(s) whose facial landmarks needs to be detected.
        face_mesh: The face landmarks detection function required to perform the landmarks detection.
    Returns:
        output_image: A copy of input image with face landmarks drawn.
        results:      The output of the facial landmarks detection on the input image.
    '''
    
    # Perform the facial landmarks detection on the image, after converting it into RGB format.
    # TODO: check why we have to put ::-1
    results = face_mesh.process(image[:,:,::-1])
    
    #---------- PRINT THE LANDMARKS ON THE IMAGE ----------
    
    # Create a copy of the input image to draw facial landmarks.
    output_image = image[:,:,::-1].copy()
    
    # Check if facial landmarks in the image are found.
    if results.multi_face_landmarks:

        # Iterate over the found faces.
        for face_landmarks in results.multi_face_landmarks:

            # Draw the facial landmarks on the output image with the face mesh tesselation
            # connections using default face mesh tesselation style.
            mp_drawing.draw_landmarks(image=output_image, landmark_list=face_landmarks,
                                      connections=mp_face_mesh.FACEMESH_TESSELATION,
                                      landmark_drawing_spec=None, 
                                      connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

            # Draw the facial landmarks on the output image with the face mesh contours
            # connections using default face mesh contours style.
            mp_drawing.draw_landmarks(image=output_image, landmark_list=face_landmarks,
                                      connections=mp_face_mesh.FACEMESH_CONTOURS,
                                      landmark_drawing_spec=None, 
                                      connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
            
    #------------------------------------------------------
    
        
    # Return the output image in BGR format and results of facial landmarks detection.
    # TODO: check why we have to put ::-1
    return np.ascontiguousarray(output_image[:,:,::-1], dtype=np.uint8), results

In [3]:
def getSize(image, face_landmarks, INDEXES):
    '''
    This function calculates the height and width of a face part utilizing its landmarks.
    Args:
        image:          The image of person(s) whose face part size is to be calculated.
        face_landmarks: The detected face landmarks of the person whose face part size is to 
                        be calculated.
        INDEXES:        The indexes of the face part landmarks, whose size is to be calculated.
    Returns:
        width:     The calculated width of the face part of the face whose landmarks were passed.
        height:    The calculated height of the face part of the face whose landmarks were passed.
        landmarks: An array of landmarks of the face part whose size is calculated.
    '''
    
    # Retrieve the height and width of the image.
    image_height, image_width, _ = image.shape
    
    # Initialize a list to store the landmarks of the face part.
    landmarks = []
    
    # Iterate over the indexes of the landmarks of the face part. 
    for INDEX in INDEXES:
        
        # Append the landmark into the list.
        landmarks.append([int(face_landmarks.landmark[INDEX].x * image_width),
                               int(face_landmarks.landmark[INDEX].y * image_height)])
    
    # Calculate the width and height of the face part.
    # TODO: Transform it to take into account the shearing --> find the 3D boundinx box and project 
    # the closest face to the screen
    _, _, width, height = cv2.boundingRect(np.array(landmarks))
    

    # Convert the list of landmarks of the face part into a numpy array.
    landmarks = np.array(landmarks)
    
    # Return the calculated width height and the landmarks of the face part.
    return width, height, landmarks

In [4]:
def isOpen(image, face_mesh_results, face_part, threshold=5):
    '''
    This function checks whether the eye or mouth of the person(s) is open, 
    utilizing its facial landmarks.
    Args:
        image:             The image of person(s) whose an eye or mouth is to be checked.
        face_mesh_results: The output of the facial landmarks detection on the image.
        face_part:         The name of the face part that is required to check: MOUTH, RIGHT EYE, LEFT EYE
        threshold:         The threshold value used to check the isOpen condition.
    Returns:
        status:       A dictionary containing isOpen statuses of the face part of all the 
                      detected faces.  
    '''
    
    # Retrieve the height and width of the image.
    image_height, image_width, _ = image.shape

    
    # Create a dictionary to store the isOpen status of the face part of all the detected faces.
    status={}
    
    # Check if the face part is mouth.
    if face_part == 'MOUTH':
        
        # Get the indexes of the mouth.
        INDEXES = mp_face_mesh.FACEMESH_LIPS
        
    # Check if the face part is left eye.    
    elif face_part == 'LEFT EYE':
        
        # Get the indexes of the left eye.
        INDEXES = mp_face_mesh.FACEMESH_LEFT_EYE
    
    # Check if the face part is right eye.    
    elif face_part == 'RIGHT EYE':
        
        # Get the indexes of the right eye.
        INDEXES = mp_face_mesh.FACEMESH_RIGHT_EYE 
           
    # Otherwise return nothing.
    else:
        return
    
    # Convert the indexes of the landmarks of the face part into a list.
    # TODO: Rewrite it in a more natural way OR undestand it + Is it necessaray to create it as a list?
    INDEXES_LIST = set(list(itertools.chain(*INDEXES)))
    
    # Iterate over the found faces.
    for face_no, face_landmarks in enumerate(face_mesh_results.multi_face_landmarks):
        
        # Get the height of the face part.
        _, height, _ = getSize(image, face_landmarks, INDEXES_LIST)
        
        # Get the height of the whole face.
        face_oval = set(list(itertools.chain(*mp_face_mesh.FACEMESH_FACE_OVAL)))
        _, face_height, _ = getSize(image, face_landmarks, face_oval)
        
        # Check if the face part is open.
        if (height/face_height)*100 > threshold:
            
            # Set status of the face part to open.
            status[face_no] = 'OPEN'
        
        # Otherwise.
        else:
            # Set status of the face part to close.
            status[face_no] = 'CLOSE'
    
    # Otherwise
    else:
        
        # Return the output image and the isOpen statuses of the face part of each detected face.
        return status

In [5]:
def boundingBox(face_landmarks, INDEXES, required_size):
    
    landmarks = []
    for index in INDEXES:
        landmarks.append([face_landmarks.landmark[index].x, 
                          face_landmarks.landmark[index].y, 
                          face_landmarks.landmark[index].z])
    
    # Create the oriented bounding box
    o3d_landmarks = o3d.utility.Vector3dVector(np.array(landmarks))
    o3d_bbox = o3d.geometry.OrientedBoundingBox.create_from_points(o3d_landmarks)
    

    #width, height, depth = o3d_bbox.get_axis_aligned_bounding_box().get_extent()
    #print("(" + str(width) + "," + str(height) + ")")
    
    # Get box points and center point
    box_points = np.asarray(o3d_bbox.get_box_points())
    box_center = np.asarray(o3d_bbox.get_center())
    
    # TODO : Improve the retrieval of points + do I have to put them in the right order here?
    
    # Retrieve forward points on the left
    #left_points = []
    #for p in box_points:
    #    if p[0] < box_center[0]:
    #        left_points.append(list(p))
    #left_points = sorted(left_points, key=lambda l: l[2])[:2]
    #left_points = sorted(left_points, key=lambda l: l[1])
    #print(left_points)
    
    # Retrieve forward points on the right
    #right_points = []
    #for p in box_points:
    #    if p[0] > box_center[0]:
    #        right_points.append(list(p))
    #right_points = sorted(right_points, key=lambda l: l[2])[:2]
    #right_points = sorted(right_points, key=lambda l: l[1], reverse=True)
    
    # Combine points --> Retrieve the forward rectangle of the bounding box
    #forward_points = np.array(left_points + right_points)
    #print(forward_points)
    
    #forward_points = box_points[[2, 0, 1, 7], :]
    #print(forward_points)
    #print(forward_points)
    
    # Project in global space and center 
    # TODO: maybe put them back in the right order here ???
    R = o3d_bbox.R
    R_inv = np.linalg.inv(R)
    
    forward_point = box_points[2, :]
    forward_point_projected = np.matmul(R_inv, forward_point - box_center)
    
    #forward_points_projected = []
    #for p in forward_points:
    #    p_translated = p - box_center # Put the origin of the local space at the origin of the global space
    #    p_rotated = np.matmul(R_inv, p_translated) # Rotate to be in the global space
    #    forward_points_projected.append(p_rotated)
    #forward_points_projected = np.array(forward_points_projected)
    #print(forward_points_projected)
    
    # Apply transformations (depends on the filter type)
    required_width, required_height = required_size
    forward_points_transformed = []
    #forward_points_transformed.append([- required_width / 2, required_height / 2, forward_points_projected[0, 2]])
    #forward_points_transformed.append([- required_width / 2, - required_height / 2, forward_points_projected[1, 2]])
    #forward_points_transformed.append([required_width / 2, - required_height / 2, forward_points_projected[2, 2]])
    #forward_points_transformed.append([required_width / 2, required_height / 2, forward_points_projected[3, 2]])
    
    # Je crois que pour la bouche ça tourne de 90° et ca fait de la merde du coup
    forward_points_transformed.append([- required_width / 2, required_height / 2, forward_point_projected[2]])
    forward_points_transformed.append([- required_width / 2, - required_height / 2, forward_point_projected[2]])
    forward_points_transformed.append([required_width / 2, - required_height / 2, forward_point_projected[2]])
    forward_points_transformed.append([required_width / 2, required_height / 2, forward_point_projected[2]])
    
    forward_points = []
    for p in forward_points_transformed:
        p_rotated = np.matmul(R, p) # Rotate back
        p_translated = p_rotated + box_center # Translate back
        forward_points.append(p_translated)
    forward_points = sorted(forward_points, key=lambda l: l[0])
    forward_points[2:] = sorted(forward_points[2:], key=lambda l: l[1], reverse = True)
    forward_points[:2] = sorted(forward_points[:2], key=lambda l: l[1])
    forward_points = np.array(forward_points)
    # TODO : remettre les points dans le bon ordre
    
    
    return o3d_bbox, box_center, forward_points


In [6]:
def drawFilter(image, filter_img, face_landmarks, INDEXES, required_size):
    
    filter_img_height, filter_img_width, _  = filter_img.shape
    image_height, image_width, _ = image.shape
    
    required_width = required_size[0]
    required_height = required_size[1]
    
    # Get 3d bounding box and fit the filter image in it
    required_height_norm = required_height / image_height
    required_width_norm = required_width / image_width
    o3d_bbox, box_center_array, forward_points = boundingBox(face_landmarks, INDEXES, 
                                                             [required_width_norm, required_height_norm])

    #print(forward_points)

    pts1 = np.array([[0, 0], [0, filter_img_height], 
                     [filter_img_width, filter_img_height], [filter_img_width, 0]])
    pts2 = []
    for point in forward_points:
        relative_x = int(point[0] * image.shape[1])
        relative_y = int(point[1] * image.shape[0])

        pts2.append([relative_x, relative_y])
    pts2 = np.array(pts2)

    h, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC,5.0)

    im1Reg = cv2.warpPerspective(filter_img, h, (image_width, image_height)) # Filter image in perspective

    _, filter_img_mask = cv2.threshold(cv2.cvtColor(im1Reg, cv2.COLOR_BGR2GRAY),
                                   25, 255, cv2.THRESH_BINARY_INV) # Inverse filter image mask

    filter_img_mask = np.expand_dims(filter_img_mask, axis=2)
    filter_img_mask = np.repeat(filter_img_mask, 3, axis=2)

    resultant_image = cv2.bitwise_and(image, filter_img_mask) # Image with black pixel at filter image position

    #Using Bitwise or to merge the two images
    annotated_image = cv2.bitwise_or(im1Reg, resultant_image)        

    relative_x = int(box_center_array[0] * frame.shape[1])
    relative_y = int(box_center_array[1] * frame.shape[0])

    cv2.circle(annotated_image, (relative_x, relative_y), radius=1, color=(0, 0, 255), thickness=7)

    # DEBUG
    i = 0
    for point in forward_points:
        relative_x = int(point[0] * frame.shape[1])
        relative_y = int(point[1] * frame.shape[0])

        cv2.circle(annotated_image, (relative_x, relative_y), radius=1, color=(0, 0 + i, 0), thickness=7)
        i+= 50

    return annotated_image

In [7]:
def overlay(image, filter_img, face_landmarks, face_part, INDEXES):
    '''
    This function will overlay a filter image over a face part of a person in the image/frame.
    Args:
        image:          The image of a person on which the filter image will be overlayed.
        filter_img:     The filter image that is needed to be overlayed on the image of the person.
        face_landmarks: The facial landmarks of the person in the image.
        face_part:      The name of the face part on which the filter image will be overlayed.
        INDEXES:        The indexes of landmarks of the face part.
        display:        A boolean value that is if set to true the function displays 
                        the annotated image and returns nothing.
    Returns:
        annotated_image: The image with the overlayed filter on the top of the specified face part.
    '''
    
    # Create a copy of the image to overlay filter image on.
    annotated_image = image.copy()
    
    # Errors can come when it resizes the filter image to a too small or a too large size .
    # So use a try block to avoid application crashing.
    try:
            
        # Get the width and height of filter image.
        filter_img_height, filter_img_width, _  = filter_img.shape
        
        # Get the width and height of the image
        image_height, image_width, _ = image.shape

        # Check if the face part is mouth.
        if face_part == 'MOUTH':
            
            # Get the height of the face part on which we will overlay the filter image.
            _, face_part_height, landmarks = getSize(image, face_landmarks, INDEXES)

            # Specify the height to which the filter image is required to be resized.
            # 2.5 can be changed depending on the size of the filter we want
            # This allows the filter to be bigger/smaller depending on both the size of the eye and the size of the aperture
            # of the eye
            required_height = int(face_part_height*1.5)
            required_width = int(filter_img_width*(required_height/filter_img_height))

            #annotated_image = drawFilter(image, filter_img, face_landmarks, INDEXES, [required_width, required_height])

        # Otherwise if the face part is an eye.
        elif face_part == 'LEFT EYE' or face_part == 'RIGHT EYE':
            
            # Get the height of the face part on which we will overlay the filter image.
            _, face_part_height, landmarks = getSize(image, face_landmarks, INDEXES)

            # Specify the height to which the filter image is required to be resized.
            # 2.5 can be changed depending on the size of the filter we want
            # This allows the filter to be bigger/smaller depending on both the size of the eye and the size of the aperture
            # of the eye
            required_height = int(face_part_height*2.5)
            required_width = int(filter_img_width*(required_height/filter_img_height))

            #annotated_image = drawFilter(image, filter_img, face_landmarks, INDEXES, [required_width, required_height])
            
        elif face_part == 'FOREHEAD':
                        
            # Get the height of the face part on which we will overlay the filter image.
            face_part_width, _, landmarks = getSize(image, face_landmarks, INDEXES)
            
            # Specify the height to which the filter image is required to be resized.
            # 2.5 can be changed depending on the size of the filter we want
            # This allows the filter to be bigger/smaller depending on both the size of the eye and the size of the aperture
            # of the eye
            required_width = int(face_part_width*2.5)
            required_height = int(filter_img_height*(required_width/filter_img_width))
            
            #annotated_image = drawFilter(image, filter_img, face_landmarks, INDEXES, [required_width, required_height])
    
        annotated_image = drawFilter(image, filter_img, face_landmarks, INDEXES, [required_width, required_height])
            
            
    # Catch and handle the error(s).
    except Exception as e:
        print(e)
        pass
            
    # Return the annotated image.
    return annotated_image

In [8]:
# Initialize the mediapipe face mesh class.
mp_face_mesh = mp.solutions.face_mesh

# Setup the face landmarks function for videos.
# TODO: Here works for only one face --> Make it work for multiple faces (see overlay call in main function)
face_mesh_videos = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, 
                                         min_detection_confidence=0.5,min_tracking_confidence=0.3)

In [9]:
# Initialize the mediapipe drawing class.
mp_drawing = mp.solutions.drawing_utils

# Initialize the mediapipe drawing styles class.
mp_drawing_styles = mp.solutions.drawing_styles

In [10]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0)

# Set camera resolution
camera_video.set(3,1280)
camera_video.set(4,960)

# Create named window for resizing purposes.
cv2.namedWindow('Face Filter', cv2.WINDOW_NORMAL)

# Read the left and right eyes images.
# TODO: Create an interactive window to let the user select the wanted filters
left_eye = cv2.imread('data/left_eye_cupcake.png')
right_eye = cv2.imread('data/right_eye_cupcake.png')

crown = cv2.imread('data/crown.png')

# Initialize the VideoCapture object to read from the smoke animation video stored in the disk.
# TODO: Same as for images --> Create a dictionary to store all images and videos?
animation = cv2.VideoCapture('data/rainbow_animation1.mp4')

# Set the smoke animation video frame counter to zero.
animation_frame_counter = 0

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
    
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if frame is not read properly then continue to the next iteration to read
    # the next frame.
    if not ok:
        continue
        
    # Get the width and height of the image
    image_height, image_width, _ = frame.shape
        
    # Read a frame from smoke animation video
    _, animation_frame = animation.read()
    
    # Increment the smoke animation video frame counter.
    animation_frame_counter += 1
    
    # Check if the current frame is the last frame of the animation video.
    if animation_frame_counter == animation.get(cv2.CAP_PROP_FRAME_COUNT):     
        
        # Set the current frame position to first frame to restart the video.
        animation.set(cv2.CAP_PROP_POS_FRAMES, 0)
        
        # Set the animation video frame counter to zero.
        animation_frame_counter = 0
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)
    
    # Perform Face landmarks detection.
    _, face_mesh_results = detectFacialLandmarks(frame, face_mesh_videos)
    
    
    # Check if facial landmarks are found.
    if face_mesh_results.multi_face_landmarks:        
        # Get the mouth isOpen status of the person in the frame.
        mouth_status = isOpen(frame, face_mesh_results, 'MOUTH', 
                                     threshold=15)
        
        # Get the left eye isOpen status of the person in the frame.
        left_eye_status = isOpen(frame, face_mesh_results, 'LEFT EYE', 
                                        threshold=4.5)
        
        # Get the right eye isOpen status of the person in the frame.
        right_eye_status = isOpen(frame, face_mesh_results, 'RIGHT EYE', 
                                         threshold=4.5)
        
        # Iterate over the found faces.
        for face_num, face_landmarks in enumerate(face_mesh_results.multi_face_landmarks):
            
            # Check if the left eye of the face is open.
            if left_eye_status[face_num] == 'OPEN':
                
                # Get the width and height of filter image.
                filter_img_height, filter_img_width, _  = left_eye.shape
                
                # Convert the indexes of the landmarks of the face part into a list.
                # TODO: Rewrite it in a more natural way OR undestand it + Is it necessaray to create it as a list?
                left_eye_landmarks = set(list(itertools.chain(*mp_face_mesh.FACEMESH_LEFT_EYE)))
                
                            # Get the height of the face part on which we will overlay the filter image.
                _, face_part_height, landmarks = getSize(frame, face_landmarks, left_eye_landmarks)

                # Specify the height to which the filter image is required to be resized.
                # 2.5 can be changed depending on the size of the filter we want
                # This allows the filter to be bigger/smaller depending on both the size of the eye and the size of the aperture
                # of the eye
                required_height = int(face_part_height*2.5)
                required_width = int(filter_img_width*(required_height/filter_img_height))
                
                frame = drawFilter(frame, left_eye, face_landmarks, 
                                   left_eye_landmarks, [required_width, required_height])
                
            
            # Check if the right eye of the face is open.
            if right_eye_status[face_num] == 'OPEN':
                
                # Get the width and height of filter image.
                filter_img_height, filter_img_width, _  = right_eye.shape
                
                # Convert the indexes of the landmarks of the face part into a list.
                # TODO: Rewrite it in a more natural way OR undestand it + Is it necessaray to create it as a list?
                right_eye_landmarks = set(list(itertools.chain(*mp_face_mesh.FACEMESH_RIGHT_EYE)))
                
                # Get the height of the face part on which we will overlay the filter image.
                _, face_part_height, landmarks = getSize(frame, face_landmarks, right_eye_landmarks)

                # Specify the height to which the filter image is required to be resized.
                # 2.5 can be changed depending on the size of the filter we want
                # This allows the filter to be bigger/smaller depending on both the size of the eye and the size of the aperture
                # of the eye
                required_height = int(face_part_height*2.5)
                required_width = int(filter_img_width*(required_height/filter_img_height))
                
                frame = drawFilter(frame, right_eye, face_landmarks, 
                                   right_eye_landmarks, [required_width, required_height])
                            
            # Check if the mouth of the face is open.
            if mouth_status[face_num] == 'OPEN':
                
                # Get the width and height of filter image.
                filter_img_height, filter_img_width, _  = animation_frame.shape
                
                # Convert the indexes of the landmarks of the face part into a list.
                # TODO: Rewrite it in a more natural way OR undestand it + Is it necessaray to create it as a list?
                lips_landmarks = set(list(itertools.chain(*mp_face_mesh.FACEMESH_LIPS)))
                
                # Get the height of the face part on which we will overlay the filter image.
                face_part_width, face_part_height, landmarks = getSize(frame, face_landmarks, lips_landmarks)
                
                # Specify the height to which the filter image is required to be resized.
                # 2.5 can be changed depending on the size of the filter we want
                # This allows the filter to be bigger/smaller depending on both the size of the eye and the size of the aperture
                # of the eye
                #required_height = int(face_part_height*1.5)
                #required_width = int(filter_img_width*(required_height/filter_img_height))
                required_width = int(face_part_width*0.7)
                required_height = int(filter_img_height*(required_width/filter_img_width))
                
                frame = drawFilter(frame, animation_frame, face_landmarks, 
                                   lips_landmarks, [required_width, required_height])
            
            # FOREHEAD
            # Get the width and height of filter image.
            filter_img_height, filter_img_width, _  = crown.shape
            
            forehead_landmarks = [103, 67, 109, 10, 338, 297, 332]
            
            # Get the height of the face part on which we will overlay the filter image.
            face_part_width,_, landmarks = getSize(frame, face_landmarks, forehead_landmarks)
                
            # Specify the height to which the filter image is required to be resized.
            # 2.5 can be changed depending on the size of the filter we want
            # This allows the filter to be bigger/smaller depending on both the size of the eye and the size of the aperture
            # of the eye
            required_width = int(face_part_width*2.5)
            required_height = int(filter_img_height*(required_width/filter_img_width))
            
            frame = drawFilter(frame, crown, face_landmarks, 
                                   forehead_landmarks, [required_width, required_height])
        
    
    # Display the frame.
    cv2.imshow('Face Filter', frame)
    
    # Wait for 1ms. If a key is pressed, retreive the ASCII code of the key.
    k = cv2.waitKey(1) & 0xFF    
    
    # Check if 'ESC' is pressed and break the loop.
    if(k == 27):
        break

# Release the VideoCapture Object and close the windows.                  
camera_video.release()
cv2.destroyAllWindows()

In [11]:
right_eye = cv2.imread('data/right_eye_cupcake.png')

filter_img_height, filter_img_width, _  = right_eye.shape

pts1 = np.array([[0, 0], [0, filter_img_height], [filter_img_width, filter_img_height], [filter_img_width, 0]])
pts2 = np.array([[0, 0], [0, filter_img_height], [filter_img_width, filter_img_height], [filter_img_width, 0]])

h, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC,5.0)

print(h)

print(filter_img_height, filter_img_width)
print()

[[ 1.00000000e+00  0.00000000e+00 -2.46139224e-14]
 [ 0.00000000e+00  1.00000000e+00 -1.47683534e-13]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
463 396



In [12]:
x = np.array([[[1], [2]], [[3], [4]]])
print(x.shape)
print(x)


x = np.repeat(x, 3, axis=2)
print(x.shape)

print(x)


(2, 2, 1)
[[[1]
  [2]]

 [[3]
  [4]]]
(2, 2, 3)
[[[1 1 1]
  [2 2 2]]

 [[3 3 3]
  [4 4 4]]]
